In [1]:
import sys
from pathlib import Path
import torch
# torch.set_grad_enabled(False)

pwd = Path('.').absolute()
sys.path.append('../')
from models.library import qres34m


d:\libraries\anaconda3\envs\pt112env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialize model.

For FLOPs computation, $\lambda$ is arbitrary since it does not affect FLOPs.

In [2]:
model = qres34m(lmb=2048)
model.eval()
model._flops_mode = True

input_shape = (3, 256, 256)
inputs = (torch.randn(1, *input_shape), )

Pytorch profiler

In [3]:
import torch.profiler as tp

with tp.profile(activities=[tp.ProfilerActivity.CPU], with_flops=True) as prof:
    model(*inputs)
torch_flops = sum([event.flops for event in prof.events()]) / 2
torch_param = sum([p.numel() for p in model.parameters()])

print(f'torch estimated FLOPs (MACs) = {torch_flops/1e9:.3g}B, parameters = {torch_param/1e6:.3f}M')


torch estimated FLOPs (MACs) = 23.2B, parameters = 34.037M


THOP: PyTorch-OpCounter

https://github.com/Lyken17/pytorch-OpCounter

In [4]:
from thop import profile, clever_format
thop_macs, thop_params = profile(model, inputs=inputs, verbose=False)

print(f'thop estimated FLOPs (MACs) = {thop_macs/1e9:.3g}B, parameters = {thop_params/1e6:.3f}M')

thop estimated FLOPs (MACs) = 23.3B, parameters = 34.019M


flops-counter.pytorch

https://github.com/sovrasov/flops-counter.pytorch

In [5]:
from ptflops import get_model_complexity_info

ptfl_macs, ptfl_params = get_model_complexity_info(model, input_shape, as_strings=False, verbose=False)

print(f'ptflops estimated FLOPs (MACs) = {ptfl_macs/1e9:.3g}B, parameters = {ptfl_params/1e6:.3f}M')

HierarchicalVAE(
  33.98 M, 99.845% Params, 23.29 GMac, 100.000% MACs, 
  (encoder): BottomUpEncoder(
    15.84 M, 46.535% Params, 10.71 GMac, 46.008% MACs, 
    (enc_blocks): ModuleList(
      15.84 M, 46.535% Params, 10.71 GMac, 46.008% MACs, 
      (0): Conv2d(9.41 k, 0.028% Params, 38.54 MMac, 0.165% MACs, 3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): MyConvNeXtBlock(
        157.63 k, 0.463% Params, 644.87 MMac, 2.769% MACs, 
        (conv_dw): Conv2d(9.6 k, 0.028% Params, 39.32 MMac, 0.169% MACs, 192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
        (norm): LayerNorm(0, 0.000% Params, 0.0 Mac, 0.000% MACs, (192,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          148.03 k, 0.435% Params, 605.55 MMac, 2.601% MACs, 
          (fc1): Linear(74.11 k, 0.218% Params, 301.99 MMac, 1.297% MACs, in_features=192, out_features=384, bias=True)
          (act): GELU(0, 0.000% Params, 1.57 MMac, 0.007% MACs, approximate=none)
          (drop1)